In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import sys
import os
import pandas as pd

In [2]:
from analysis_functions import *

Calculate temperature using color index data

In [12]:
def apparent_mag(flux, zeropoint):
    return -2.5 * np.log10(flux) + zeropoint

In [6]:
filters = ['g', 'r', 'i']
filter_zeropoints = [21.65063765247328, 21.025287319733366, 20.03855919309204] # instrumental zeropoint (mag), for g, r, i
color_index_types = ['g-r', 'r-i', 'g-i']

In [28]:
# for van Maanen's star
star = 'van_Maanen_'
radius = 8 # pixels
VM_fluxes = []
x_star, y_star = 1189, 1173

file_root = r'C:\Users\Willi\Desktop\134L Images\good_fits'
for filter in filters:
    file = os.path.join(file_root, star + f"{filter}" + '.fits')
    # print(file)

    hdul = fits.open(file)
    data = hdul[1].data

    y, x = np.indices(data.shape)  # Create coordinate grids for the image
    distance = np.sqrt((x - x_star)**2 + (y - y_star)**2)  # Distance of each pixel from the star center

    aperture_mask = distance <= radius

    background_mask = (distance > radius + 1) & (distance <= radius + 10)
    background_values = data[background_mask]  # Pixels outside the aperture
    background_median = np.median(background_values)  # Median background level

    aperture_flux = np.sum(data[aperture_mask] - background_median)

    VM_fluxes.append(aperture_flux)

print(VM_fluxes)

[170625.5, 123303.49, 51127.43]


In [10]:
def calculate_flux(file_root, star_name, x_position, y_position, radius):
    fluxes = []
    for filter in filters:
        file = os.path.join(file_root, star_name + f"{filter}" + '.fits')

        hdul = fits.open(file)
        data = hdul[1].data

        y, x = np.indices(data.shape)  # Create coordinate grids for the image
        distance = np.sqrt((x - x_position)**2 + (y - y_position)**2)  # Distance of each pixel from the star center

        aperture_mask = distance <= radius

        background_mask = (distance > radius + 1) & (distance <= radius + 10)
        background_values = data[background_mask]  # Pixels outside the aperture
        background_median = np.median(background_values)  # Median background level

        aperture_flux = np.sum(data[aperture_mask] - background_median)

        fluxes.append(aperture_flux)

    return fluxes

In [11]:
calculate_flux(r'C:\Users\Willi\Desktop\134L Images\good_fits', 'van_Maanen_', 1189, 1173, 8)

[170625.5, 123303.49, 51127.43]

In [47]:
VM_index_g = mag(VM_fluxes[0], filter_zeropoints[0])
VM_index_r = mag(VM_fluxes[1], filter_zeropoints[1])
VM_index_i = mag(VM_fluxes[2], filter_zeropoints[2])
print(VM_index_g, VM_index_r, VM_index_i)

VM_gr = VM_index_g - VM_index_r
VM_ri = VM_index_r - VM_index_i
VM_gi = VM_index_g - VM_index_i
print(VM_gr, VM_ri, VM_gi)

VM_indices = [VM_gr, VM_ri, VM_gi]

8.570528294639416 8.297849108292326 8.266924375968628
0.27267918634709076 0.030924732323697413 0.3036039186707882


In [33]:
color_index_type = 'g-r'
temperature = calculate_temperature(VM_gr, color_index_type, data)
print(f"The temperature for van Maanen's star color index {color_index} with color index type {color_index_type} is {temperature} K")

The temperature for van Maanen's star color index 0.65 with color index type g-r is 7600 K


In [22]:
color_index_type = 'r-i'
temperature = calculate_temperature(VM_ri, color_index_type, data)
print(f"The temperature for van Maanen's star color index {color_index} with color index type {color_index_type} is {temperature} K")

The temperature for van Maanen's star color index 0.65 with color index type r-i is 10000 K


In [23]:
color_index_type = 'g-i'
temperature = calculate_temperature(VM_gi, color_index_type, data)
print(f"The temperature for van Maanen's star color index {color_index} with color index type {color_index_type} is {temperature} K")

The temperature for van Maanen's star color index 0.65 with color index type g-i is 8200 K


In [24]:
print('Average temp for van Maanen: ' , (7600+10000+8200)/3)

Average temp for van Maanen:  8600.0


In [48]:
temperatures = []
for i in range(3):
    temperature = calculate_temperature(VM_indices[i], color_index_types[i], data)
    temperatures.append(temperature)
    # print(f"The temperature for LAWD 8 color index {color_index} with color index type {color_index_type} is {temperature} K")

print('Average temp for van Maanen: ' , np.mean(temperatures))

Average temp for van Maanen:  8600.0


Now for LAWD 8

In [27]:
# for LAWD 8
star = 'LAWD_8_'
radius = 7 # pixels
lawd8_fluxes = []
x_star, y_star = 1208, 1216

file_root = r'C:\Users\Willi\Desktop\134L Images\good_fits'
for filter in filters:
    file = os.path.join(file_root, star + f"{filter}" + '.fits')
    # print(file)

    hdul = fits.open(file)
    data = hdul[1].data

    y, x = np.indices(data.shape)  # Create coordinate grids for the image
    distance = np.sqrt((x - x_star)**2 + (y - y_star)**2)  # Distance of each pixel from the star center

    aperture_mask = distance <= radius

    background_mask = (distance > radius + 1) & (distance <= radius + 10)
    background_values = data[background_mask]  # Pixels outside the aperture
    background_median = np.median(background_values)  # Median background level

    aperture_flux = np.sum(data[aperture_mask] - background_median)

    lawd8_fluxes.append(aperture_flux)

print(lawd8_fluxes)

[67049.05, 38889.344, 11649.363]


In [45]:
lawd8_index_g = mag(lawd8_fluxes[0], filter_zeropoints[0])
lawd8_index_r = mag(lawd8_fluxes[1], filter_zeropoints[1])
lawd8_index_i = mag(lawd8_fluxes[2], filter_zeropoints[2])
print(lawd8_index_g, lawd8_index_r, lawd8_index_i)

lawd8_gr = lawd8_index_g - lawd8_index_r
lawd8_ri = lawd8_index_r - lawd8_index_i
lawd8_gi = lawd8_index_g - lawd8_index_i
print(lawd8_gr, lawd8_ri, lawd8_gi)
lawd8_indices = [lawd8_gr, lawd8_ri, lawd8_gi]

9.584656025962659 9.55071132333444 9.872803444343262
0.03394470262821869 -0.3220921210088221 -0.2881474183806034


In [46]:
temperatures = []
for i in range(3):
    temperature = calculate_temperature(lawd8_indices[i], color_index_types[i], data)
    temperatures.append(temperature)
    # print(f"The temperature for LAWD 8 color index {color_index} with color index type {color_index_type} is {temperature} K")

print('Average temp for LAWD 8: ' , np.mean(temperatures))

Average temp for LAWD 8:  11400.0


Try with just the imported functions.

van Maanen's Star

In [3]:
file_root = r'C:\Users\Willi\Desktop\134L Images\good_fits'

In [16]:
omi_fluxes = calculate_flux(file_root, 'omi02EriB_', 1213, 1233, 7)
print(omi_fluxes)
omi_apparent_mag = np.zeros(3)
for i in range(3):
    omi_apparent_mag[i] = apparent_mag(omi_fluxes[i], filter_zeropoints[i])

print(omi_apparent_mag)

[10478.992, 15511.519, 21200.893]
[11.599839   10.54865163  9.22267413]


In [7]:
calculate_all(file_root, 'WD0621-376_', 1166, 1259, 8)

([42506.223, 16040.875, 3895.8025],
 array([10.08052978, 10.57227696, 10.93378067]),
 array([100000., 100000., 100000.]),
 100000.0,
 0.0)

In [6]:
calculate_all(file_root, 'LHS_253_', 1251, 1175, 8)

([58707.164, 35686.727, 10611.516],
 array([9.64278643, 9.62425715, 9.71333189]),
 array([10500., 14900., 11700.]),
 12366.666666666666,
 1857.1184369578827)

In [5]:
calculate_all(file_root, 'LAWD_8_', 1208, 1216, 8)

([67477.87, 39389.742, 11928.922],
 array([9.54811516, 9.59673402, 9.74001694]),
 array([11700., 19700., 13500.]),
 14966.666666666666,
 3426.6926069057063)

In [4]:
calculate_all(file_root, 'van_Maanen_', 1189, 1173, 8)

([170625.5, 123303.49, 51127.43],
 array([8.57052829, 8.29784911, 8.26692438]),
 array([7600., 9900., 8300.]),
 8600.0,
 962.6352718795767)

In [8]:
calculate_all(file_root, 'omi02EriB_', 1213, 1233, 7)

([10478.992, 15511.519, 21200.893],
 array([11.64759626, 10.63086215,  9.10950793]),
 array([4300., 2200., 3200.]),
 3233.3333333333335,
 857.6453553512405)